In [2]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


In [9]:
%%sql
SELECT
    p.name,
    p.designation,
    vc.ispassportstamplocation,
    vc.name as visitor_center_name
FROM nps_public_data.parks p
LEFT JOIN nps_public_data.visitorcenters vc
    ON p.parkcode = vc.parkcode
WHERE 1 = 1
    AND p.designation = 'National Monument'
    AND vc.ispassportstamplocation
LIMIT 3

,name,designation,isPassportStampLocation,visitor_center_name
0,Statue Of Liberty,National Monument,1,Liberty Island Information Center
1,Tuzigoot,National Monument,1,Tuzigoot Visitor Center
2,Timpanogos Cave,National Monument,1,Timpanogos Cave Visitor Center


In [10]:
%%sql
WITH filter_in_join AS (
    SELECT
        p.name,
        vc.name as visitor_center_name
    FROM nps_public_data.parks p
    INNER JOIN nps_public_data.visitorcenters vc
        ON p.parkcode = vc.parkcode
), filter_in_where AS (
    SELECT
        p.name,
        vc.name as visitor_center_name
    FROM nps_public_data.parks p
    LEFT JOIN nps_public_data.visitorcenters vc
        ON p.parkcode = vc.parkcode
    WHERE vc.parkcode IS NOT NULL
)
SELECT
    COUNT(*) as ct
FROM filter_in_join

UNION ALL

SELECT
    COUNT(*) as ct
FROM filter_in_where


,ct
0,703
1,703


OPERATORS:

1. Comparisons (`>`, `<`, `=`)
2. `BETWEEN`
3. `IN`
4. `IS NULL`
5. `LIKE` & `ILIKE` // `REGEXP`


`ILIKE`, `LIKE`, and `REGEXP` are all useful when pattern matching is at play.

In [11]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart > '2024-01-01'
    AND recurrencedatestart < '2024-01-23'
ORDER BY RANDOM()
LIMIT 2


,title,parkfullname,category,isfree,description
0,Acadian Cultural Center - Louisiana Talks & Tales,Jean Lafitte National Historical Park and Pres...,Regular Event,True,"<p>Join a ranger to learn about the history, c..."


In [12]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart BETWEEN '2024-01-01' AND '2024-01-23' -- between is INCLUSIVE 
ORDER BY RANDOM()
LIMIT 2

,title,parkfullname,category,isfree,description
0,Acadian Cultural Center - Louisiana Talks & Tales,Jean Lafitte National Historical Park and Pres...,Regular Event,True,"<p>Join a ranger to learn about the history, c..."
1,Afternoon Stroll,Joshua Tree National Park,Regular Event,True,<p>Join a ranger for a 0.4 mile (0.6 km) guide...


In [13]:
%%sql
SELECT
    'between' as f,
    COUNT(*) as ct
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart BETWEEN '2024-01-01' AND '2024-01-23'
GROUP BY f

UNION ALL

SELECT
    'greater than' as f,
    COUNT(*) as ct
FROM nps_public_data.events e
WHERE 1 = 1
    AND recurrencedatestart > '2024-01-01'
    AND recurrencedatestart < '2024-01-23'
GROUP BY f

,f,ct
0,between,3
1,greater than,1


In [14]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    -- Fetch events with dates in January _or_ March
    AND (
            (recurrencedatestart BETWEEN '2024-01-01' AND '2024-01-31') OR
            (recurrencedatestart BETWEEN '2024-03-01' AND '2024-03-31')
    ) 
ORDER BY RANDOM()
LIMIT 2

,title,parkfullname,category,isfree,description
0,Afternoon Stroll,Joshua Tree National Park,Regular Event,True,<p>Join a ranger for a 0.4 mile (0.6 km) guide...
1,10 am to 4 pm: Village — Ahkima and Marie Hony...,Grand Canyon National Park,Regular Event,True,"<p>This week, the Cultural Demonstration Progr..."


In [15]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND title LIKE '%Stroll%'
LIMIT 5

,title,parkfullname,category,isfree,description
0,Afternoon Stroll,Joshua Tree National Park,Regular Event,True,<p>Join a ranger for a 0.4 mile (0.6 km) guide...


In [17]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND title LIKE '%hike%' -- LIKE is case sensetive
LIMIT 5

,title,parkfullname,category,isfree,description


In [18]:
%%sql
SELECT
    title,
    parkfullname,
    category,
    isfree,
    description
FROM nps_public_data.events e
WHERE 1 = 1
    AND title ILIKE '%hike%' -- ILIKE is case insensitive 
LIMIT 5

,title,parkfullname,category,isfree,description
0,A Hike Through The (Cactus) Forest (East Distr...,Saguaro National Park,Regular Event,True,<p>Let your interest reach new heights and joi...


In [19]:
%%sql 
SELECT 
    name,
    managedByOrganization,
FROM nps_public_data.parkinglots
LIMIT 10

,name,managedByOrganization
0,Corn Creek Road Parking Area,Nevada Department of Transportation
1,Glen Echo Park and Clara Barton National Histo...,National Park Service
2,Glen Echo Park and Clara Barton National Histo...,National Park Service
3,Kaloko Fishpond Parking Area,National Park Service
4,Main Visitor Parking Lot,NPS
5,Natchez Visitor Center Parking Lot,NPS
6,Parking lot Remodel and Construction,NPS
7,Quarai Oversized Vehicle Parking,NPS
8,Quarry Visitor Center Parking Lot,Dinosaur National Monument
9,Shark Valley Parking Lot,NPS


In [ ]:
%%sql 
SELECT 
    CASE WHEN name ILIKE '%visitor%' THEN 'Visitor Center'
         WHEN name ILIKE '%parking%' THEN 'Parking Lot'
         ELSE 'Other'
    END as type,
    IF(managedByOrganization ILIKE '%NPS%', 'National Park Service', managedByOrganization) as managed_by,

FROM nps_public_data.parkinglots
LIMIT 10

,type,managed_by
0,Parking Lot,Corn Creek Road Parking Area
1,Parking Lot,Glen Echo Park and Clara Barton National Histo...
2,Parking Lot,Glen Echo Park and Clara Barton National Histo...
3,Parking Lot,Kaloko Fishpond Parking Area
4,Visitor Center,National Park Service
5,Visitor Center,National Park Service
6,Parking Lot,National Park Service
7,Parking Lot,National Park Service
8,Visitor Center,Quarry Visitor Center Parking Lot
9,Parking Lot,National Park Service


In [3]:
%%sql
SELECT fullname, states FROM nps_public_data.parks

,fullName,states
0,Federal Hall National Memorial,NY
1,Lewis & Clark National Historic Trail,"IA,ID,IL,IN,KS,KY,MO,MT,NE,ND,OH,OR,PA,SD,WA,WV"
2,National Capital Parks-East,DC
3,Adams National Historical Park,MA
4,George Washington Memorial Parkway,"DC,MD,VA"
...,...,...
466,Navajo National Monument,AZ
467,Cabrillo National Monument,CA
468,Golden Spike National Historical Park,UT
469,Fort Union Trading Post National Historic Site,"MT,ND"


In [4]:
%%sql
-- Which parks are fully or partially in Utah?
WITH park_states AS (
    SELECT 
        fullname,
        states AS states_string, 
        split(states, ',') ::string[] AS states_list
    FROM nps_public_data.parks p
    )
SELECT 
    * 
FROM park_states
WHERE list_contains(states_list, 'UT')
LIMIT 5

,fullName,states_string,states_list
0,Cedar Breaks National Monument,UT,[UT]
1,Arches National Park,UT,[UT]
2,Bryce Canyon National Park,UT,[UT]
3,California National Historic Trail,"CA,CO,ID,KS,MO,NE,NV,OR,UT,WY","[CA, CO, ID, KS, MO, NE, NV, OR, UT, WY]"
4,Canyonlands National Park,UT,[UT]


In [5]:
%%sql
-- Which parks are both in Utah and Wyoming?
WITH park_states AS (
    SELECT 
        fullname,
        split(states, ',') ::string[] AS states_list
    FROM nps_public_data.parks p
    )
SELECT 
    * 
FROM park_states
WHERE list_has_all(states_list, ['UT', 'WY'])

,fullName,states_list
0,California National Historic Trail,"[CA, CO, ID, KS, MO, NE, NV, OR, UT, WY]"
1,Mormon Pioneer National Historic Trail,"[IL, IA, NE, UT, WY]"
2,Pony Express National Historic Trail,"[CA, CO, KS, MO, NE, NV, UT, WY]"


In [6]:
%%sql
-- Which parks are in Utah and/or Wyoming?
WITH park_states AS (
    SELECT 
        fullname,
        split(states, ',') ::string[] AS states_list
    FROM nps_public_data.parks p
    )
SELECT 
    * 
FROM park_states
WHERE list_has_any(states_list, ['UT', 'WY'])
LIMIT 5

,fullName,states_list
0,Cedar Breaks National Monument,[UT]
1,Yellowstone National Park,"[ID, MT, WY]"
2,Arches National Park,[UT]
3,Bryce Canyon National Park,[UT]
4,California National Historic Trail,"[CA, CO, ID, KS, MO, NE, NV, OR, UT, WY]"


In [7]:
%%sql
SELECT 
    fullname,
    states,
    description
FROM nps_public_data.parks p
WHERE name IN ('Arches', 'Bryce Canyon', 'Zion')

,fullName,states,description
0,Arches National Park,UT,"Discover a landscape of contrasting colors, la..."
1,Bryce Canyon National Park,UT,Hoodoos (irregular columns of rock) exist on e...
2,Zion National Park,UT,Follow the paths where people have walked for ...


In [8]:
%%sql
SELECT
    fullname,
    states
FROM nps_public_data.parks
ORDER BY fullname DESC
LIMIT 5

,fullName,states
0,Zion National Park,UT
1,Yukon - Charley Rivers National Preserve,AK
2,Yucca House National Monument,CO
3,Yosemite National Park,CA
4,Yorktown Battlefield Part of Colonial National...,VA


In [9]:
%%sql
SELECT
    DISTINCT states
FROM nps_public_data.parks
LIMIT 5

,states
0,MI
1,WI
2,TX
3,CO
4,OR
